In [1]:
%autosave 0

Autosave disabled


In [2]:
import pandas as pd
import numpy as np
import re
import pprint
from collections import OrderedDict

In [3]:
! head ftes_lorvec.out

#RPA-results:
#Nucleus: SN 146
#Excitation: 1 -
#Total number of pairs: 2188
#Number of ph-/ah-pairs: 1554/634
#Maximal excitation-energy for particle-hole-pairs: 200
#Maximal excitation-energy for antiparticle-hole-pairs: 2000
#2 imaginary eigenvalues
#Listing imaginary eigenvalues:
#Imaginary part: 4.383495e-01


In [4]:
146 - 50

96

In [5]:
# 'energy', 'transition_strength'
#  names=['col1', 'col2']
df_lorvec = pd.read_csv('ftes_lorvec.out', delim_whitespace=True, comment='#', skip_blank_lines=True,
            header=None, names=['col1', 'col2'])

In [6]:
df_lorvec = df_lorvec[(df_lorvec.col1 >= 0.1) & (df_lorvec.col1 <= 30)] # MeV

In [7]:
df_lorvec.head()

,col1,col2
10,0.10,0.135900
11,0.11,0.136607
12,0.12,0.137321
13,0.13,0.138042
14,0.14,0.138770


In [8]:
every_10 = df_lorvec.iloc[::10, :]
every_10.reset_index(drop=True, inplace=True)
df = every_10.iloc[:4, :].T
print(df.to_string())
print()
print(df.index)

           0        1         2         3
col1  0.1000  0.20000  0.300000  0.400000
col2  0.1359  0.14329  0.151455  0.160519
Index(['col1', 'col2'], dtype='object')


In [37]:
# print(table.to_string())
print()
# print(table.index)

                  0        1         2         3
Z  A                                            
50 100 col1  0.1000  0.20000  0.300000  0.400000
       col2  0.1359  0.14329  0.151455  0.160519

MultiIndex(levels=[['50'], ['100', '101'], ['col1', 'col2']],
           labels=[[0, 0], [0, 0], [0, 1]],
           names=['Z', 'A', None])


In [10]:
myidx = pd.MultiIndex(levels=[['50'], ['100'], ['col1', 'col2']],
                      labels=[[0, 0], [0, 0], [0, 1]],
                       names=['Z', 'A', None])

In [11]:
print(df.reindex(index=myidx).to_string())

              0   1   2   3
Z  A                       
50 100 col1 NaN NaN NaN NaN
       col2 NaN NaN NaN NaN


In [12]:
stocks = pd.read_csv('http://bit.ly/smallstocks')

In [13]:
stocks.groupby('Symbol').Close.mean()

Symbol
AAPL    112.856667
CSCO     31.480000
MSFT     57.433333
Name: Close, dtype: float64

In [14]:
ser = stocks.groupby(['Symbol', 'Date']).Close.mean()

In [15]:
ser.index

MultiIndex(levels=[['AAPL', 'CSCO', 'MSFT'], ['2016-10-03', '2016-10-04', '2016-10-05']],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=['Symbol', 'Date'])

In [16]:
ser.unstack()

Date,2016-10-03,2016-10-04,2016-10-05
Symbol,,,
AAPL,112.52,113.00,113.05
CSCO,31.50,31.35,31.59
MSFT,57.42,57.24,57.64


In [17]:
df = stocks.pivot_table(values='Close', index='Symbol', columns='Date')

In [18]:
stocks.set_index(['Symbol', 'Date'], inplace=True)

In [19]:
ser.loc[:, '2016-10-03']

Symbol
AAPL    112.52
CSCO     31.50
MSFT     57.42
Name: Close, dtype: float64

In [20]:
stocks.index

MultiIndex(levels=[['AAPL', 'CSCO', 'MSFT'], ['2016-10-03', '2016-10-04', '2016-10-05']],
           labels=[[1, 0, 2, 0, 2, 1, 2, 1, 0], [0, 0, 0, 1, 1, 1, 2, 2, 2]],
           names=['Symbol', 'Date'])

In [21]:
stocks.sort_index(inplace=True)

In [22]:
def to_float(iterable):
    return (float(val) for val in iterable)

In [23]:
class ConfigurationSyntaxError(Exception):
    pass

In [24]:
def fn_to_dict(fname):
    z_from_fname = int(''.join(filter(lambda c: not c.isalpha(), fname)))
    contents = open(fname).read()
    
    new_line = re.compile('\n[\s\r]+?\n')
    blocks = new_line.split(contents)
    
    assert blocks[-1].strip() == '', "Last line not empty!"
    assert len(blocks[:-1]) == 82, "Not right number of blocks!"
    
    mydict = OrderedDict()
    mydict[z_from_fname] =  OrderedDict()
    
    minA, maxA = 400, 0
    for blk in blocks[:-1]: # except last line
        block = blk.splitlines()
    
        nucleus_header = block[0].split()
        Z, A = (int(val) for val in nucleus_header[1::2])
        if (nucleus_header[0::2] != ['Z=', 'A=']) or (Z != z_from_fname):
            raise ConfigurationSyntaxError('Wrong header inside %s!' % fname)
        if int(A) < minA:
            minA = int(A)
        if int(A) > maxA:
            maxA = int(A)
        
        col_header = block[1].split()
        if col_header[:2] != ['U[MeV]', 'fE1[mb/MeV]']:
            raise ConfigurationSyntaxError('Wrong header inside %s!' % fname)

        columns = (row.split() for row in block[2:])
        c1_vals, c2_vals = zip(*columns)

        d = OrderedDict()
        d[A] = OrderedDict(U=list(to_float(c1_vals)),
                         fE1=list(to_float(c2_vals)))
        mydict[Z].update(d)
        
    print(f"Amax - Amin = {maxA - minA}, {len(blocks[:-1])} blocks")
    return mydict, minA, maxA

In [25]:
md, a, b = fn_to_dict('z050')
print(f"Amin = {a}, Amax = {b}")

Amax - Amin = 88, 82 blocks
Amin = 90, Amax = 178


In [26]:
def dict_to_df(ordered_nested_dict):
    reform = {(firstKey, secondKey, thirdKey): values for firstKey, middleDict in ordered_nested_dict.items() for secondKey, innerdict in middleDict.items() for thirdKey, values in innerdict.items()}
    df = pd.DataFrame(reform)
    df = df.T # transpose
    df.index.names=['Z', 'A', None]
    return df

In [27]:
# pprint.pprint(md)

In [28]:
df = dict_to_df(md)
df.head()

0         1         2         3         4         5    \
Z  A                                                                    
50 90 U    0.100000  0.200000  0.300000  0.400000  0.500000  0.600000   
      fE1  0.006291  0.006776  0.007300  0.007862  0.008465  0.009107   
   91 U    0.100000  0.200000  0.300000  0.400000  0.500000  0.600000   
      fE1  0.006350  0.006840  0.007369  0.007937  0.008545  0.009193   
   92 U    0.100000  0.200000  0.300000  0.400000  0.500000  0.600000   

                6        7        8        9     ...         290      291  \
Z  A                                             ...                        
50 90 U    0.700000  0.80000  0.90000  1.00000   ...     29.1000  29.2000   
      fE1  0.009790  0.01051  0.01128  0.01209   ...      0.9121   0.9004   
   91 U    0.700000  0.80000  0.90000  1.00000   ...     29.1000  29.2000   
      fE1  0.009882  0.01061  0.01138  0.01220   ...      0.9619   0.9492   
   92 U    0.700000  0.80000  0.90000  1.00000   ...     29.1000  29.2000   

               292      293      294      295      296      297      298  \
Z  A                                                                       
50 90 U    29.3000  29.4000  29.5000  29.6000  29.7000  29.8000  29.9000   
      fE1   0.8889   0.8775   0.8662   0.8550   0.8439   0.8329   0.8220   
   91 U    29.3000  29.4000  29.5000  29.6000  29.7000  29.8000  29.9000   
      fE1   0.9365   0.9238   0.9111   0.8984   0.8858   0.8731   0.8605   
   92 U    29.3000  29.4000  29.5000  29.6000  29.7000  29.8000  29.9000   

               299  
Z  A                
50 90 U    30.0000  
      fE1   0.8110  
   91 U    30.0000  
      fE1   0.8479  
   92 U    30.0000  

[5 rows x 300 columns]

In [29]:
md_tst = {'50': {'100': {'col1': (0.100,
                              0.200,
                              0.300,
                              0.400),
                     'col2': (0.1359,
                              0.14329,
                              0.151455,
                              0.160519)},
             '101': {'col1': (0.100,
                              0.200,
                              0.300,
                              0.400),
                     'col2': (6.510E-03,
                              7.011E-03,
                              7.553E-03,
                              8.134E-03)}
            }
     }

In [30]:
df_tst = dict_to_df(md_tst)
df_tst

0         1         2         3
Z  A                                              
50 100 col1  0.10000  0.200000  0.300000  0.400000
       col2  0.13590  0.143290  0.151455  0.160519
   101 col1  0.10000  0.200000  0.300000  0.400000
       col2  0.00651  0.007011  0.007553  0.008134

In [31]:
df_tst = df_tst.T
pprint.pprint(df_tst.to_dict('list')) # dict is OK
df_tst = df_tst.T
print(df_tst.index) # index is OK

{('50', '100', 'col1'): [0.1, 0.2, 0.3, 0.4],
 ('50', '100', 'col2'): [0.1359, 0.14329, 0.151455, 0.160519],
 ('50', '101', 'col1'): [0.1, 0.2, 0.3, 0.4],
 ('50', '101', 'col2'): [0.00651, 0.007011, 0.007553, 0.008134]}
MultiIndex(levels=[['50'], ['100', '101'], ['col1', 'col2']],
           labels=[[0, 0, 0, 0], [0, 0, 1, 1], [0, 1, 0, 1]],
           names=['Z', 'A', None])


In [32]:
table = df_tst.loc[('50', '100', ['col1', 'col2']), :]
table

0        1         2         3
Z  A                                            
50 100 col1  0.1000  0.20000  0.300000  0.400000
       col2  0.1359  0.14329  0.151455  0.160519

In [33]:
table.T

Z   50          
A  100          
  col1      col2
0  0.1  0.135900
1  0.2  0.143290
2  0.3  0.151455
3  0.4  0.160519

In [34]:
stocks

Close    Volume
Symbol Date                        
AAPL   2016-10-03  112.52  21701800
       2016-10-04  113.00  29736800
       2016-10-05  113.05  21453100
CSCO   2016-10-03   31.50  14070500
       2016-10-04   31.35  18460400
       2016-10-05   31.59  11808600
MSFT   2016-10-03   57.42  19189500
       2016-10-04   57.24  20085900
       2016-10-05   57.64  16726400

In [35]:
stocks.loc[('AAPL', '2016-10-03'), :] # rows, columns
stocks.loc[('AAPL', '2016-10-03'), 'Close']
stocks.loc[(['AAPL', 'MSFT'], '2016-10-03'), :]
stocks.loc[(['AAPL', 'MSFT'], '2016-10-03'), 'Close']
stocks.loc[('AAPL', ['2016-10-03', '2016-10-04']), 'Close']
stocks.loc[(slice(None), ['2016-10-03', '2016-10-04']), :]

Close    Volume
Symbol Date                        
AAPL   2016-10-03  112.52  21701800
       2016-10-04  113.00  29736800
CSCO   2016-10-03   31.50  14070500
       2016-10-04   31.35  18460400
MSFT   2016-10-03   57.42  19189500
       2016-10-04   57.24  20085900